### Automated backwards optimization

Automate the discovery of strategy rules.

In [ ]:
generate indicator combinations
    * [2, 4, 8, etc]
    * [adx, ]

for i in indicators:
    for start_date in range(first_date, last_date):
        for end_date in range(start_date, last_date):
            for lower_limit in range(min(i), max(i), some_step):
                for upper_limit in range(lower_limit + 1?, max(i), some_step):

                    compute profitability in range

                    save results

for i, j in indicator pairs:
    for start_date in range(first_date, last_date):
        for end_date in range(start_date, last_date):
            for lower_limit_i in range(min(i), max(i), some_step):
                for upper_limit_i in range(lower_limit + 1?, max(i), some_step):                    
                    for lower_limit_j in range(min(j), max(j), some_step):
                        for upper_limit_j in range(lower_limit + 1?, max(j), some_step):
                            compute profitability in range

                            save results


In [1]:
import datetime 
import json
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from dateutil import rrule
import numpy as np

import pandas as pd
import tqdm
from matplotlib import pyplot
import seaborn as sns
import dask.dataframe as dd

# TIMEFRAME = 'daily'
TIMEFRAME = 'weekly'

# FRAMEWORK = 'dask'
FRAMEWORK = 'pandas'

FIRST_DATE = datetime.datetime(2000, 1, 1)
LAST_DATE = datetime.datetime(2020, 12, 31)

FORWARD_OPT_RESULTS_JSON = f'results/{TIMEFRAME}/forward_opt_results.json'
FORWARD_OPT_RESULTS_JSON = 'test.json'
BACKWARD_OPT_RESULTS_JSON = f'results/{TIMEFRAME}/backward_opt_results.json'
BACKWARD_OPT_RESULTS_JSON = 'test_b.json'

# The minimum number of trades in a period for it to be considered.
MIN_TRADES_REQUIRED = 10

In [ ]:
metrics_options = ["adx", "ppo", "stochastic"]


if TIMEFRAME == 'daily':
    size_options = [8, 16, 32]    
    path = 'dataset/daily/'
    column_names = [
        "uid",
        "ticker",
        "date",
        "price_open",
        "maperiod",
        "rsi_open_period",
        "adx8",
        "adx16",
        "adx32",
        "ppo8",
        "ppo16",
        "ppo32",
        "stochastic8",
        "stochastic16",
        "stochastic32",
        "price_sell",
        "days_ago_close_period",
        "rsi_close_period",
    ]
elif TIMEFRAME == 'weekly':
    size_options = [3, 6, 9]    
    size_options = [8, 16, 32]  # TODO: remove this after dataset & forward opt jobs run    
    path = 'dataset/weekly/'    
    column_names = [
        "uid",
        "ticker",
        "date",
        "price_open",
        "maperiod",
        "rsi_open_period",
        "adx3",
        "adx6",
        "adx9",
        "ppo3",
        "ppo6",
        "ppo9",
        "stochastic3",
        "stochastic6",
        "stochastic9",
        "price_sell",
        "days_ago_close_period",
        "rsi_close_period",
    ]

if FRAMEWORK == 'pandas':
    d = pd
    f = f"{path}/all_results.csv"
elif FRAMEWORK == 'dask':
    d = dd
    f = f"{path}/*.csv"

FULL_DF = d.read_csv(
    f,
    names=column_names,
)

def get_full_df():
    return d.read_csv(
        f,
        names=column_names,
    )

In [4]:
# TODO: remove this after dataset jobs run
def get_full_df():
    if TIMEFRAME == 'daily':
        path = 'dataset/daily/'
    elif TIMEFRAME == 'weekly':
        path = 'dataset/weekly/'

    if FRAMEWORK == 'pandas':
        d = pd
        f = f"{path}/all_results.csv"
    elif FRAMEWORK == 'dask':
        d = dd
        f = f"{path}/*.csv"

    # Read CSV file and assign the right column names
    # Comment the first two rows if you don't want to use the entire dataset, and instead just work on a sample
    return d.read_csv(
        f,
        names=[
            "uid",
            "ticker",
            "date",
            "price_open",
            "maperiod",
            "rsi_open_period",
            #         'rsi_close_period',        # not for 'efficient_results'
            # 'days_ago_close_period'  # not for 'efficient_results'
            "adx8",
            "adx16",
            "adx32",
            "adx64",
            "adx128",
            "adx256",
            "adx512",
            "mdi8",
            "mdi16",
            "mdi32",
            "mdi64",
            "mdi128",
            "mdi256",
            "mdi512",
            "pdi8",
            "pdi16",
            "pdi32",
            "pdi64",
            "pdi128",
            "pdi256",
            "pdi512",
            "ppo8",
            "ppo16",
            "ppo32",
            "ppo64",
            "ppo128",
            "ppo256",
            "ppo512",
            "stochastic8",
            "stochastic16",
            "stochastic32",
            "stochastic64",
            "stochastic128",
            "stochastic256",
            "stochastic512",
            "price_sell",
            "days_ago_close_period",
            "rsi_close_period",
        ],
    )

In [5]:
get_full_df()

,uid,ticker,date,price_open,maperiod,rsi_open_period,adx8,adx16,adx32,adx64,...,stochastic8,stochastic16,stochastic32,stochastic64,stochastic128,stochastic256,stochastic512,price_sell,days_ago_close_period,rsi_close_period
0,1c4fc506-bbc0-11eb-ab02-bbaa3923f212,AAPL,2017-08-10,38.31,30,1,44.578880,41.340568,24.143496,12.151689,...,91.195066,91.891470,96.497608,97.730483,97.758649,98.517226,98.913740,38.23,0,0.000000
1,1c4fc506-bbc0-11eb-ab02-bbaa3923f212,AAPL,2017-08-10,38.31,30,1,44.578880,41.340568,24.143496,12.151689,...,91.195066,91.891470,96.497608,97.730483,97.758649,98.517226,98.913740,39.03,1,100.000000
2,1c4fc506-bbc0-11eb-ab02-bbaa3923f212,AAPL,2017-08-10,38.31,30,1,44.578880,41.340568,24.143496,12.151689,...,91.195066,91.891470,96.497608,97.730483,97.758649,98.517226,98.913740,38.69,2,0.000000
3,1c4fc506-bbc0-11eb-ab02-bbaa3923f212,AAPL,2017-08-10,38.31,30,1,44.578880,41.340568,24.143496,12.151689,...,91.195066,91.891470,96.497608,97.730483,97.758649,98.517226,98.913740,38.15,3,0.000000
4,1c4fc506-bbc0-11eb-ab02-bbaa3923f212,AAPL,2017-08-10,38.31,30,1,44.578880,41.340568,24.143496,12.151689,...,91.195066,91.891470,96.497608,97.730483,97.758649,98.517226,98.913740,37.29,4,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772322,fd1856e1-bc09-11eb-ab34-bbaa3923f212,YUM,2019-10-24,100.47,59,6,36.435305,34.418995,25.757593,17.597855,...,16.592687,21.311603,49.272405,71.912945,78.422879,84.796825,88.466934,95.65,1,12.720749
772323,fd1856e1-bc09-11eb-ab34-bbaa3923f212,YUM,2019-10-24,100.47,59,6,36.435305,34.418995,25.757593,17.597855,...,16.592687,21.311603,49.272405,71.912945,78.422879,84.796825,88.466934,95.40,2,12.452106
772324,fd1856e1-bc09-11eb-ab34-bbaa3923f212,YUM,2019-10-24,100.47,59,6,36.435305,34.418995,25.757593,17.597855,...,16.592687,21.311603,49.272405,71.912945,78.422879,84.796825,88.466934,98.27,3,32.182258
772325,fd1856e1-bc09-11eb-ab34-bbaa3923f212,YUM,2019-10-24,100.47,59,6,36.435305,34.418995,25.757593,17.597855,...,16.592687,21.311603,49.272405,71.912945,78.422879,84.796825,88.466934,96.86,4,28.407906


In [10]:
def get_forward_opt_json():
    with open(FORWARD_OPT_RESULTS_JSON, 'r') as f:
        return json.loads(f.read())

FORWARD_OPT_JSON = get_forward_opt_json()

def get_filtered_df_by_key(key, range_name, func_type):
    optimal_args = FORWARD_OPT_JSON[key][range_name][func_type]
    df = get_full_df()
  
    for k, v in optimal_args.items():
        df = df[df[k] == v]
        
    return df
    
# get_filtered_df_by_key('17-10', '6-month', 'mean')

,uid,ticker,date,price_open,maperiod,rsi_open_period,adx8,adx16,adx32,adx64,...,stochastic8,stochastic16,stochastic32,stochastic64,stochastic128,stochastic256,stochastic512,price_sell,days_ago_close_period,rsi_close_period
6623,3d529800-bbc0-11eb-ab02-bbaa3923f212,AAPL,2018-11-08,45.31,59,7,33.213761,24.225852,22.255029,16.928672,...,27.306433,39.094456,61.819870,67.349532,80.563434,83.264143,87.345749,38.26,5,18.000822
6629,3d529801-bbc0-11eb-ab02-bbaa3923f212,AAPL,2020-03-12,61.09,59,7,58.993497,42.834072,25.855337,16.671882,...,34.404338,34.748647,60.761046,72.207867,72.207867,78.524681,82.519131,71.26,5,56.143288
15197,2652d3c1-bbbc-11eb-ab02-bbaa3923f212,ABT,2019-09-26,77.39,59,7,33.264725,16.719478,16.907394,16.874210,...,23.089063,37.797460,60.707397,76.743324,85.910530,87.789761,90.562422,82.28,5,60.126661
19421,c0b16746-bbc0-11eb-ab03-bbaa3923f212,ADM,2018-11-15,41.78,59,7,26.908414,24.164956,16.034789,7.884667,...,16.244863,16.244863,45.712787,64.228856,66.409717,78.734102,84.487594,37.79,5,22.048429
28643,3ce7e9e3-bbbe-11eb-ab02-bbaa3923f212,AEE,2018-01-04,51.67,59,7,35.991438,29.718409,26.610958,20.990490,...,4.129539,11.836735,42.839275,65.485013,77.395152,82.257032,87.060239,49.76,5,32.543195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702039,f5722ebb-bc04-11eb-ab2e-bbaa3923f212,USB,2020-01-23,50.89,59,7,34.612179,16.658456,8.629510,5.562027,...,8.807919,29.032272,42.908147,67.730692,67.730692,77.335395,86.072842,34.38,5,7.084677
713759,07513959-bc06-11eb-ab2f-bbaa3923f212,VLO,2020-01-23,78.98,59,7,20.145857,14.495630,11.395618,9.844903,...,13.029879,38.272871,62.116892,66.843419,50.018044,61.517088,72.152408,50.30,5,6.789414
730600,8b0e438a-bc08-11eb-ab32-bbaa3923f212,WFC,2020-01-09,46.21,59,7,38.463241,22.908293,13.893790,9.580778,...,31.239450,60.690571,73.805310,76.875000,50.000000,54.696133,76.726585,42.66,5,19.831795
749594,284432ce-bc07-11eb-ab2f-bbaa3923f212,WMT,2018-02-22,84.08,59,7,51.047357,41.796456,26.206160,13.423479,...,13.477528,29.782106,56.887568,66.412566,72.730683,72.730683,78.702212,80.73,5,24.635852


In [11]:
keys = [end_date.strftime('%y-%m') for end_date in rrule.rrule(rrule.MONTHLY, dtstart=FIRST_DATE, until=LAST_DATE)]
range_values = [
    '6-month',
    '12-month',
    'max',
]
func_values = [
    'mean',
    'order'
]

In [12]:
def get_backward_opt_json():
    with open(BACKWARD_OPT_RESULTS_JSON, 'r') as f:
        return json.loads(f.read())


BACKWARD_OPT_JSON = get_backward_opt_json()   


def is_pair_parsed(key, range_val, func_val):
    if key not in BACKWARD_OPT_JSON:
        return False
    
    if range_val not in BACKWARD_OPT_JSON[key]:
        return False
    
    if func_val not in BACKWARD_OPT_JSON[key][range_val]:
        return False
    
    return True
    

# for key, range_val, func_val in list(product(keys, range_values, func_values)):
#     if is_pair_parsed(key, range_val, func_val) is False:
#         print(f"Pair {key} - {range_val} - {func_val} is NOT already parsed!")

Pair 07-02 - 6-month - mean is NOT already parsed!
Pair 07-02 - 6-month - order is NOT already parsed!
Pair 07-02 - 12-month - mean is NOT already parsed!
Pair 07-02 - 12-month - order is NOT already parsed!
Pair 07-02 - max - mean is NOT already parsed!
Pair 07-02 - max - order is NOT already parsed!
Pair 07-03 - 6-month - mean is NOT already parsed!
Pair 07-03 - 6-month - order is NOT already parsed!
Pair 07-03 - 12-month - mean is NOT already parsed!
Pair 07-03 - 12-month - order is NOT already parsed!
Pair 07-03 - max - mean is NOT already parsed!
Pair 07-03 - max - order is NOT already parsed!
Pair 07-04 - 6-month - mean is NOT already parsed!
Pair 07-04 - 6-month - order is NOT already parsed!
Pair 07-04 - 12-month - mean is NOT already parsed!
Pair 07-04 - 12-month - order is NOT already parsed!
Pair 07-04 - max - mean is NOT already parsed!
Pair 07-04 - max - order is NOT already parsed!
Pair 07-05 - 6-month - mean is NOT already parsed!
Pair 07-05 - 6-month - order is NOT alre

In [13]:
def save_pair_result(key, range_val, func_val, optimal_res):
    with open(BACKWARD_OPT_RESULTS_JSON, 'r') as f:
        backward_opt_json = json.loads(f.read())
        
    backward_opt_json[key] = backward_opt_json.get(key, {})
    backward_opt_json[key][range_val] = backward_opt_json[key].get(range_val, {})
    backward_opt_json[key][range_val][func_val] = optimal_res
    
    with open(BACKWARD_OPT_RESULTS_JSON, 'w') as f:
        f.write(json.dumps(backward_opt_json))

# save_pair_result('00-01', '6-month', 'mean', 1)
# is_pair_parsed('00-01', '6-month', 'mean')

In [16]:
def mean_optimal_func(tmp_df):
    max_value = tmp_df["perc_change"].mean()

    if FRAMEWORK == 'dask':
        max_value = max_value.compute()

    return max_value


def order_optimal_func(tmp_df_groupby):
    def outcome_computation(x):
        l = []
        for _, perc_change in x.sort_values(['date'])['perc_change'].iteritems():
            l.append(perc_change)

        results = []
        for i in range(len(l)):
            res = 1
            for j in range(i, len(l)):
                res += (res * l[j]) / 100
            results.append(res)

        return sum(results) / len(results)

    if FRAMEWORK == 'pandas':
        outcome_series = tmp_df_groupby.apply(outcome_computation)
    else:
        outcome_series = tmp_df_groupby.apply(outcome_computation, meta=('float'))

    res = outcome_series.idxmax()
    return res if FRAMEWORK == 'pandas' else res.compute()


In [17]:
singles = [f"{x[0]}{x[1]}" for x in list(product(metrics_options, size_options))]
doubles = list(combinations(indicators, 2))

def single_compute_optimal_res(df, func):
    upper_barrier_results = []
    lower_barrier_results = []
    
    for indicator in singles:
        max_val, min_val = int(df[indicator].max()), int(df[indicator].min())

        for barrier in range(min_val, max_val, 2):
            filtered_df = df[df[indicator] <= barrier]
            outcome = func(filtered_df)
            if outcome and outcome not in [np.nan]:
                upper_barrier_results.append([indicator, barrier, float(outcome)])
    
            filtered_df = df[df[indicator] >= barrier]
            outcome = func(filtered_df)
            if outcome and outcome not in [np.nan]:  # TODO: why 'outcome and ' is needed?
                lower_barrier_results.append([indicator, barrier, float(outcome)])

    upper_barrier_df = pd.DataFrame(
        np.array(upper_barrier_results),
       columns=['indicator', 'barrier', 'value']
    )
    upper_barrier_res = upper_barrier_df[upper_barrier_df['value'] == upper_barrier_df['value'].max()]

    lower_barrier_df = pd.DataFrame(
        np.array(lower_barrier_results),
       columns=['indicator', 'barrier', 'value']
    )
    lower_barrier_res = lower_barrier_df[lower_barrier_df['value'] == lower_barrier_df['value'].max()]

    return {
        'upper': {
            'indicator': upper_barrier_res.iloc[0]['indicator'],
            'barrier': int(upper_barrier_res.iloc[0]['barrier'])
        },
        'lower': {
            'indicator': lower_barrier_res.iloc[0]['indicator'],
            'barrier': int(lower_barrier_res.iloc[0]['barrier'])
        }
    }
    
    
def double_compute_optimal_res(df, func): # TODO
      pass
    

def compute_optimal_res(df):
    result = {}
    for name, func in [('mean', mean_optimal_func),]:# ('order', order_optimal_func)]:  # TODO
        result[name] = {
            'single': single_compute_optimal_res(df, func),
#             'double': double_compute_optimal_res(df, func),
        }

    return result

In [14]:
all_pairs = list(product(keys, range_values, func_values))

unparsed_pairs = [
    (key, range_val, func_val)
    for key, range_val, func_val in all_pairs
    if not is_pair_parsed(key, range_val, func_val)
]

len(unparsed_pairs)

984

In [15]:
# TODO: run this after the dataset and forward opt things finish
for key, range_val, func_val in unparsed_pairs:
    try:
        df = get_filtered_df_by_key(key, range_val, func_val)
        
        df["perc_change"] = (df["price_sell"] - df["price_open"]) / df["price_open"] * 100
        df = df.drop(["price_sell", "price_open"], axis=1)
    
    except KeyError:
        print(f'Key not in forward opt json {key} - {range_val} - {func_val}')
        continue
    
    size = df.shape[0] if FRAMEWORK == 'pandas' else df.shape[0].compute()

    if size < MIN_TRADES_REQUIRED:
        print(f'Not enough trades for {key} - {range_val} - {func_val}')
        optimal_res = None
    else:
        optimal_res = compute_optimal_res(df)
    
    print(f'{key} - {range_val} - {func_val}')
    print(optimal_res)
    save_pair_result(key, range_val, func_val, optimal_res)
    

Key not in forward opt json 07-02 - 6-month - mean
Key not in forward opt json 07-02 - 6-month - order
Key not in forward opt json 07-02 - 12-month - mean
Key not in forward opt json 07-02 - 12-month - order
Key not in forward opt json 07-02 - max - mean
Key not in forward opt json 07-02 - max - order
Key not in forward opt json 07-03 - 6-month - mean
Key not in forward opt json 07-03 - 6-month - order
Key not in forward opt json 07-03 - 12-month - mean
Key not in forward opt json 07-03 - 12-month - order
Key not in forward opt json 07-03 - max - mean
Key not in forward opt json 07-03 - max - order
Key not in forward opt json 07-04 - 6-month - mean
Key not in forward opt json 07-04 - 6-month - order
Key not in forward opt json 07-04 - 12-month - mean
Key not in forward opt json 07-04 - 12-month - order
Key not in forward opt json 07-04 - max - mean
Key not in forward opt json 07-04 - max - order
Key not in forward opt json 07-05 - 6-month - mean
Key not in forward opt json 07-05 - 6-mo